In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import datetime

In [2]:

# https://open.spotify.com/playlist/0i2S0eEdftTrmLKueMWUKX?si=7f10c20af4d04ea1
# english: https://open.spotify.com/playlist/2oE3flopAvGvpv9QqkhV5Q
# hindi:
#
#
 #https://open.spotify.com/playlist/6A546Y17RhQ6MrjGIec68L?si=e0790235d2c04e63


###importing all the necessary libraries





In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id='ac1f41b0f93c45ac8af29623bdf94e0a', client_secret='f2460ff1a77e4c6db5eca633dde46d3b')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


# playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX0XUfTFmNBRM?si=7934cecad3f3439c"
playlist_link = input("Enter the link: ")

playlist_URI = playlist_link.split("/")[-1].split("?")[0]

songs = sp.playlist_tracks(playlist_URI, market='IN')['items']  #will give a list of songs (meta-data)

In [4]:
song_name = []
song_id = []
song_popu = []
song_added_date = []
song_release_date = []
for song in songs:
    song_name.append(song['track']['name'])
    song_id.append(song['track']['id'])
    song_popu.append(song['track']['popularity'])
    song_added_date.append(song['added_at'])
    song_release_date.append(song['track']['album']['release_date'])
 
release_year = map(lambda x: int(x[:4]), song_release_date)

In [5]:
#combining the data
import pandas as pd

df = pd.DataFrame({
    'name':song_name,
    'popularity':song_popu,
    'date_added':song_added_date,
    'release_year':release_year
    })

In [6]:
#audio features

features = sp.audio_features(song_id)

feat_names = list(sp.audio_features(song_id)[0].keys())

for row in range(len(features)):
    for col in range(len(feat_names)):
        df.loc[row, feat_names[col]] = features[row][feat_names[col]]
# df

In [7]:
#artists
artists_col = []
for i in range(len(songs)):
    all_artists = songs[i]['track']['artists']
    artists = []
    for a in all_artists:
        artists.append(a['name'])
    artists_col.append(artists)
df['artists'] = artists_col

In [8]:
df['artists']

0               [Alan Walker, Ava Max]
1     [Raaban, Tungevaag, Luana Kiara]
2                           [Maroon 5]
3          [Wiz Khalifa, Charlie Puth]
4            [DJ Snake, Justin Bieber]
                    ...               
94                          [Dua Lipa]
95                         [ZAYN, Sia]
96                      [Jaymes Young]
97                             [KALEO]
98                  [Eminem, Dina Rae]
Name: artists, Length: 99, dtype: object

In [9]:
df['popularity'] = list(map(lambda x: x // 5, df['popularity']))

for i in range(0,21):
    df[f"popu|{i}"] = [0] * len(df['name'])

for i in range(1980, datetime.date.today().year + 1):
    df[f"year|{i}"] = [0] * len(df['name'])

df_year = pd.get_dummies(df['release_year'])
df_popu = pd.get_dummies(df['popularity'])
df_year.columns = map(lambda x: 'year' + '|' + str(x), df_year.columns)
df_popu.columns = map(lambda x: 'popu' + '|' + str(x), df_popu.columns)
df_popu.head()

for col in df_popu.columns:
    df[col] = df_popu[col]
for col in df_year.columns:
    df[col] = df_year[col]

In [10]:
new_df = df.copy()
new_df['artists']

0               [Alan Walker, Ava Max]
1     [Raaban, Tungevaag, Luana Kiara]
2                           [Maroon 5]
3          [Wiz Khalifa, Charlie Puth]
4            [DJ Snake, Justin Bieber]
                    ...               
94                          [Dua Lipa]
95                         [ZAYN, Sia]
96                      [Jaymes Young]
97                             [KALEO]
98                  [Eminem, Dina Rae]
Name: artists, Length: 99, dtype: object

In [11]:
artists_excel = pd.read_excel('datasets/artists_names.xlsx')

excel_df = pd.read_csv("datasets/final_data.csv")

In [12]:
# for converting the stringed list to again list

# print(excel_df.loc[i, 'artists'])
art = []
for i in range(len(excel_df['name'])):
    b = 0
    ele = ''
    new_l = []
    for ch in excel_df.loc[i, 'artists']:
        if b == 1:
            if ch == "'":
                b = 0
                new_l.append(ele)
                ele = ''
            else:
                ele += ch
        elif ch == "'":
            b = 1

    art.append(new_l)
print(art)
# excel_df['artists'] = art
# excel_df['artists']
    # print(new_l)
    # print(excel_df.loc[i, 'artists'])

excel_df['artists'] = art

[['Harrdy Sandhu'], ['Badshah', 'Payal Dev'], ['Badshah', 'Nikhita Gandhi'], ['Badshah'], ['Guru Randhawa', 'Nikhita Gandhi', 'Tanishk Bagchi', 'Nora Fatehi'], ['Sachet Tandon', 'Nikhita Gandhi', 'Madhubanti Bagchi', 'Sachin-Jigar'], ['Jubin Nautiyal', 'Meet Bros.', 'Shardul Rathod'], ['Tanishk Bagchi', 'Jubin Nautiyal', 'Asees Kaur'], ['Pritam', 'Arijit Singh'], ['Vishal-Shekhar', 'Arijit Singh', 'Shilpa Rao', 'Kumaar'], ['Vishal Dadlani', 'Benny Dayal'], ['Pritam', 'Arijit Singh', 'Neha Kakkar', 'Meet Bros.'], ['Sachet Tandon', 'Dhvani Bhanushali', 'Tanishk Bagchi'], ['Hasan Shah'], ['B Praak', 'Harrdy Sandhu', 'Swasti Mehul'], ['A.R. Rahman', 'Shreya Ghoshal', 'Irshad Kamil'], ['A.R. Rahman', 'Shreya Ghoshal'], ['Vishal-Shekhar', 'Arijit Singh', 'Caralisa Monteiro', 'Jaideep Sahni'], ['Yo Yo Honey Singh'], ['Badshah'], ['Badshah', 'Neha Kakkar', 'Varun Dhawan'], ['Badshah', 'Neeti Mohan'], ['Arijit Singh'], ['Palak Muchhal'], ['Vayu', 'Aastha Gill', 'AKASA', 'Puri'], ['Arijit Singh'

In [13]:
zeros = [0] * len(new_df['name'])
extra = pd.DataFrame(zeros)
# for name in artists_excel['artists']:
#     extra[f"artist|{name}"] = 0

for name in artists_excel['artists']:
    extra[f"artist|{name}"] = 0

new_df = pd.concat([new_df, extra], axis=1)
new_df.head()
excel_df = pd.concat([excel_df, new_df])

C:\Users\pavan\AppData\Local\Temp\ipykernel_15212\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  extra[f"artist|{name}"] = 0
C:\Users\pavan\AppData\Local\Temp\ipykernel_15212\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  extra[f"artist|{name}"] = 0
C:\Users\pavan\AppData\Local\Temp\ipykernel_15212\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [14]:
# pd.concat([excel_df, new_df])

In [15]:
# artists_excel = pd.read_excel('datasets/artists_names.xlsx')

# excel_df = pd.read_excel("datasets/final_data.xlsx")


# zeros = [0] * len(new_df['name'])
# extra = pd.DataFrame(zeros)

# for name in artists_excel['artists']:
#     extra[f"artist|{name}"] = 0

# new_df = pd.concat([new_df, extra], axis=1)
# new_df.head()
# excel_df = pd.concat([excel_df, new_df])

In [16]:
print(len(excel_df), len(df))

1691 99


In [17]:
excel_df = pd.concat([excel_df, new_df])
for name in artists_excel['artists']:
    excel_df[f"artist|{name}"] = 0
excel_df.loc[:, 'year|2023':]

,year|2023,artist|Pritam,artist|Arijit Singh,artist|Amitabh Bhattacharya,artist|Kaifi Khalil,artist|Tanishk Bagchi,artist|Jubin Nautiyal,artist|Asees Kaur,artist|King,artist|Jasleen Royal,...,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [18]:
excel_df.drop([0], axis=1, inplace=True)

In [19]:
len(excel_df)

1790

In [20]:
# excel_df.head(20)
# ex = excel_df.copy()
# ex.reset_index(inplace=True)
# ex.drop(['index'], axis=1, inplace=True)
# # ex

In [21]:
# ex['artists'] = art
# excel_df = ex.copy()
# excel_df

In [22]:
excel_df = excel_df.reset_index().drop('index', axis=1)
excel_df.iloc[76, :]
# np.array(excel_df.loc[75, 'artists'])

Unnamed: 0                                                     76.0
name                       Galliyan (Unplugged) [From "Ek Villain"]
popularity                                                        9
date_added                                     2022-09-29T21:24:03Z
release_year                                                   2017
                                             ...                   
artist|Confetti                                                   0
artist|Victoria Justice                                           0
artist|Tainy                                                      0
artist|Glee Cast                                                  0
artist|Jamie Miller                                               0
Name: 76, Length: 1319, dtype: object

In [23]:
# excel_df = excel_df.reset_index()
# excel_df.drop('index', axis=1, inplace=True)

for i in range(len(excel_df['name'])):
    # print(excel_df.loc[i, 'artists'])
    for name in np.array(excel_df.loc[i, 'artists']):
        if name in np.array(artists_excel['artists']):
            excel_df.loc[i, f"artist|{name}"] = 1

# for i, row in excel_df.iterrows():
#     for name in row['artists']:
#         print(list(row['artists']))
#         # print(artists_excel['artists'])
#         if name in list(artists_excel['artists']):
#             excel_df.loc[i, f"artist|{name}"] = 1

# excel_df = excel_df.drop(0, axis=1)

In [24]:
list(excel_df.columns)

['Unnamed: 0',
 'name',
 'popularity',
 'date_added',
 'release_year',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'artists',
 'popu|0',
 'popu|1',
 'popu|2',
 'popu|3',
 'popu|4',
 'popu|5',
 'popu|6',
 'popu|7',
 'popu|8',
 'popu|9',
 'popu|10',
 'popu|11',
 'popu|12',
 'popu|13',
 'popu|14',
 'popu|15',
 'popu|16',
 'popu|17',
 'popu|18',
 'popu|19',
 'popu|20',
 'year|1980',
 'year|1981',
 'year|1982',
 'year|1983',
 'year|1984',
 'year|1985',
 'year|1986',
 'year|1987',
 'year|1988',
 'year|1989',
 'year|1990',
 'year|1991',
 'year|1992',
 'year|1993',
 'year|1994',
 'year|1995',
 'year|1996',
 'year|1997',
 'year|1998',
 'year|1999',
 'year|2000',
 'year|2001',
 'year|2002',
 'year|2003',
 'year|2004',
 'year|2005',
 'year|2006',
 'year|2007',
 'year|2008',
 'year|2009',
 'year|2010',
 'y

In [25]:
# excel_df['name|artists'] = list(map(lambda x, y: f"{x}|{str(y)}", excel_df['name'], excel_df['artists']))

In [26]:
excel_df.columns
excel_df['id']

0       1iZLpuGMr4tn1F5bZu32Kb
1       0eiX2zTStU6Cz4hjUWnUa1
2       2NTVtNcpC0i1R7LVGaCZCF
3       6iSUIEoQaZXB2gtTCTeQYv
4       1orM67pmkNc4alrZkWjYkV
                 ...          
1785    39LLxExYz6ewLAcYrzQQyP
1786    3e7sxremeOE3wTySiOhGiP
1787    1SOClUWhOi8vHZYMz3GluK
1788    0y1QJc3SJVPKJ1OvFmFqe6
1789    4woTEX1wYOTGDqNXuavlRC
Name: id, Length: 1790, dtype: object

In [27]:
excel_df = excel_df.drop_duplicates('id')
excel_df.columns

Index(['Unnamed: 0', 'name', 'popularity', 'date_added', 'release_year',
       'danceability', 'energy', 'key', 'loudness', 'mode',
       ...
       'artist|Lost Frequencies', 'artist|Miss Congeniality', 'artist|Torpe',
       'artist|Baby Tyson', 'artist|Seb Tik', 'artist|Confetti',
       'artist|Victoria Justice', 'artist|Tainy', 'artist|Glee Cast',
       'artist|Jamie Miller'],
      dtype='object', length=1319)

In [28]:
len(excel_df)

1592

In [29]:
excel_df = excel_df.reset_index().drop('index', axis=1)
excel_df.columns

Index(['Unnamed: 0', 'name', 'popularity', 'date_added', 'release_year',
       'danceability', 'energy', 'key', 'loudness', 'mode',
       ...
       'artist|Lost Frequencies', 'artist|Miss Congeniality', 'artist|Torpe',
       'artist|Baby Tyson', 'artist|Seb Tik', 'artist|Confetti',
       'artist|Victoria Justice', 'artist|Tainy', 'artist|Glee Cast',
       'artist|Jamie Miller'],
      dtype='object', length=1319)

In [30]:
# excel_df = pd.read_excel("final_data.xlsx")
# pd.concat([excel_df, new_df])
if 'year|0' in excel_df.columns:
    excel_df = excel_df.drop('year|0', axis=1)
if 'Unnamed: 0' in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0', axis=1)
if "Unnamed: 0.1" in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0.1', axis=1)
if "Unnamed: 0.2" in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0.2', axis=1)
excel_df.to_csv('datasets/final_data.csv')

In [31]:
excel_df

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,artist|Lost Frequencies,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller
0,Bijlee Bijlee,14,2022-02-12T07:13:38Z,2021,0.795,0.670,1.0,-5.313,0.0,0.1430,...,0,0,0,0,0,0,0,0,0,0
1,"Sajna, Say Yes To The Dress",13,2022-02-12T07:14:15Z,2021,0.633,0.776,9.0,-3.839,0.0,0.1730,...,0,0,0,0,0,0,0,0,0,0
2,Jugnu,14,2022-02-12T07:14:23Z,2021,0.716,0.803,10.0,-5.140,0.0,0.0445,...,0,0,0,0,0,0,0,0,0,0
3,She Move It Like,9,2022-02-12T07:14:39Z,2018,0.832,0.477,0.0,-6.949,1.0,0.0481,...,0,0,0,0,0,0,0,0,0,0
4,Naach Meri Rani,12,2022-02-12T07:14:57Z,2020,0.676,0.488,1.0,-5.091,1.0,0.1540,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,Paro - Speed Up,13,2023-03-01T14:43:16Z,2021,0.651,0.829,5.0,-5.266,0.0,0.1280,...,0,0,0,0,0,0,0,0,0,0
1588,Play Date (Lofi),10,2023-03-01T14:44:25Z,2021,0.673,0.152,11.0,-24.291,0.0,0.0331,...,0,0,0,0,0,0,0,0,0,0
1589,The Box,16,2023-03-01T14:45:10Z,2019,0.896,0.586,10.0,-6.687,0.0,0.0559,...,0,0,0,0,0,0,0,0,0,0
1590,Levitating,16,2023-03-01T14:45:27Z,2020,0.695,0.884,6.0,-2.278,0.0,0.0753,...,0,0,0,0,0,0,0,0,0,0


In [32]:
song_name = []
song_id = []
# song_popu = []
# song_added_date = []
# song_release_date = []
for song in songs:
    song_name.append(song['track']['name'])
    song_id.append(song['track']['id'])
    # song_popu.append(song['track']['popularity'])
    # song_added_date.append(song['added_at'])
    # song_release_date.append(song['track']['album']['release_date'])
    
# release_year = map(lambda x: int(x[:4]), son

In [33]:
new_df

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,artist|Lost Frequencies,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller
0,"Alone, Pt. II",15,2021-06-20T17:16:50Z,2019,0.670,0.668,3.0,-4.306,0.0,0.0422,...,0,0,0,0,0,0,0,0,0,0
1,Bad Boy (feat. Luana Kiara),13,2021-06-20T17:16:54Z,2018,0.644,0.892,0.0,-1.100,0.0,0.0567,...,0,0,0,0,0,0,0,0,0,0
2,Memories,17,2021-06-20T17:17:01Z,2021,0.775,0.327,11.0,-7.241,1.0,0.0557,...,0,0,0,0,0,0,0,0,0,0
3,See You Again (feat. Charlie Puth),17,2021-06-20T17:17:09Z,2015,0.689,0.481,10.0,-7.503,1.0,0.0815,...,0,0,0,0,0,0,0,0,0,0
4,Let Me Love You,17,2021-06-20T17:17:22Z,2016,0.649,0.716,8.0,-5.371,1.0,0.0349,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Levitating,16,2023-03-01T14:45:27Z,2020,0.695,0.884,6.0,-2.278,0.0,0.0753,...,0,0,0,0,0,0,0,0,0,0
95,Dusk Till Dawn (feat. Sia) - Radio Edit,16,2023-03-01T14:45:59Z,2018,0.259,0.437,11.0,-6.589,0.0,0.0386,...,0,0,0,0,0,0,0,0,0,0
96,Infinity,16,2023-03-01T14:46:32Z,2017,0.671,0.673,11.0,-6.941,0.0,0.0444,...,0,0,0,0,0,0,0,0,0,0
97,Way down We Go,17,2023-07-11T16:37:08Z,2016,0.489,0.505,10.0,-8.022,0.0,0.1170,...,0,0,0,0,0,0,0,0,0,0


In [34]:

import pandas as pd

data_new= pd.DataFrame({
    'name':song_name,
    # 'popularity':song_popu,
    # 'date_added':song_added_date,
    # 'release_year':release_year
    })

In [42]:
#audio features

features = sp.audio_features(song_id)

# feat_names = list(sp.audio_features(song_id)[0].keys())
feat_names = ["danceability", "energy",	"key",	"loudness",	"mode",	"speechiness",	"acousticness",	"instrumentalness",	"liveness",	"valence",	"tempo"]


for row in range(len(features)):
    for col in range(len(feat_names)):
        data_new.loc[row, feat_names[col]] = features[row][feat_names[col]]


In [43]:
data_new

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,"Alone, Pt. II",0.670,0.668,3.0,-4.306,0.0,0.0422,0.5360,0.000000,0.1330,0.6080,88.033
1,Bad Boy (feat. Luana Kiara),0.644,0.892,0.0,-1.100,0.0,0.0567,0.1300,0.000113,0.0893,0.7670,180.044
2,Memories,0.775,0.327,11.0,-7.241,1.0,0.0557,0.8410,0.000000,0.0821,0.5950,91.050
3,See You Again (feat. Charlie Puth),0.689,0.481,10.0,-7.503,1.0,0.0815,0.3690,0.000001,0.0649,0.2830,80.025
4,Let Me Love You,0.649,0.716,8.0,-5.371,1.0,0.0349,0.0863,0.000026,0.1350,0.1630,99.988
...,...,...,...,...,...,...,...,...,...,...,...,...
94,Levitating,0.695,0.884,6.0,-2.278,0.0,0.0753,0.0561,0.000000,0.2130,0.9140,103.014
95,Dusk Till Dawn (feat. Sia) - Radio Edit,0.259,0.437,11.0,-6.589,0.0,0.0386,0.1020,0.000001,0.1060,0.0951,180.042
96,Infinity,0.671,0.673,11.0,-6.941,0.0,0.0444,0.1480,0.000053,0.3040,0.4940,121.963
97,Way down We Go,0.489,0.505,10.0,-8.022,0.0,0.1170,0.5790,0.000333,0.1040,0.3370,163.255


In [44]:
data_new

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,"Alone, Pt. II",0.670,0.668,3.0,-4.306,0.0,0.0422,0.5360,0.000000,0.1330,0.6080,88.033
1,Bad Boy (feat. Luana Kiara),0.644,0.892,0.0,-1.100,0.0,0.0567,0.1300,0.000113,0.0893,0.7670,180.044
2,Memories,0.775,0.327,11.0,-7.241,1.0,0.0557,0.8410,0.000000,0.0821,0.5950,91.050
3,See You Again (feat. Charlie Puth),0.689,0.481,10.0,-7.503,1.0,0.0815,0.3690,0.000001,0.0649,0.2830,80.025
4,Let Me Love You,0.649,0.716,8.0,-5.371,1.0,0.0349,0.0863,0.000026,0.1350,0.1630,99.988
...,...,...,...,...,...,...,...,...,...,...,...,...
94,Levitating,0.695,0.884,6.0,-2.278,0.0,0.0753,0.0561,0.000000,0.2130,0.9140,103.014
95,Dusk Till Dawn (feat. Sia) - Radio Edit,0.259,0.437,11.0,-6.589,0.0,0.0386,0.1020,0.000001,0.1060,0.0951,180.042
96,Infinity,0.671,0.673,11.0,-6.941,0.0,0.0444,0.1480,0.000053,0.3040,0.4940,121.963
97,Way down We Go,0.489,0.505,10.0,-8.022,0.0,0.1170,0.5790,0.000333,0.1040,0.3370,163.255


In [45]:
###storing the name column
song_name=data_new["name"]


In [46]:
### preparing the data the prediction of emotion 
x=data_new.drop("name",axis=1)
x


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.670,0.668,3.0,-4.306,0.0,0.0422,0.5360,0.000000,0.1330,0.6080,88.033
1,0.644,0.892,0.0,-1.100,0.0,0.0567,0.1300,0.000113,0.0893,0.7670,180.044
2,0.775,0.327,11.0,-7.241,1.0,0.0557,0.8410,0.000000,0.0821,0.5950,91.050
3,0.689,0.481,10.0,-7.503,1.0,0.0815,0.3690,0.000001,0.0649,0.2830,80.025
4,0.649,0.716,8.0,-5.371,1.0,0.0349,0.0863,0.000026,0.1350,0.1630,99.988
...,...,...,...,...,...,...,...,...,...,...,...
94,0.695,0.884,6.0,-2.278,0.0,0.0753,0.0561,0.000000,0.2130,0.9140,103.014
95,0.259,0.437,11.0,-6.589,0.0,0.0386,0.1020,0.000001,0.1060,0.0951,180.042
96,0.671,0.673,11.0,-6.941,0.0,0.0444,0.1480,0.000053,0.3040,0.4940,121.963
97,0.489,0.505,10.0,-8.022,0.0,0.1170,0.5790,0.000333,0.1040,0.3370,163.255


In [48]:
import pickle 

mod=pickle.load(open("mod1.pkl","rb"))


In [49]:
y=mod.predict(x)


In [50]:
###creating the new dataframe with name and the emotion of that song 
emotion_song=pd.DataFrame([])
emotion_song["name"]=song_name
emotion_song["emotion"]=y


In [51]:
emotion_song

,name,emotion
0,"Alone, Pt. II",sad
1,Bad Boy (feat. Luana Kiara),happy
2,Memories,chill
3,See You Again (feat. Charlie Puth),chill
4,Let Me Love You,angry
...,...,...
94,Levitating,happy
95,Dusk Till Dawn (feat. Sia) - Radio Edit,sad
96,Infinity,chill
97,Way down We Go,chill


In [53]:
new_df["emotion"]=y

C:\Users\pavan\AppData\Local\Temp\ipykernel_15212\2283493151.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df["emotion"]=y


In [54]:
new_df

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller,emotion
0,"Alone, Pt. II",15,2021-06-20T17:16:50Z,2019,0.670,0.668,3.0,-4.306,0.0,0.0422,...,0,0,0,0,0,0,0,0,0,sad
1,Bad Boy (feat. Luana Kiara),13,2021-06-20T17:16:54Z,2018,0.644,0.892,0.0,-1.100,0.0,0.0567,...,0,0,0,0,0,0,0,0,0,happy
2,Memories,17,2021-06-20T17:17:01Z,2021,0.775,0.327,11.0,-7.241,1.0,0.0557,...,0,0,0,0,0,0,0,0,0,chill
3,See You Again (feat. Charlie Puth),17,2021-06-20T17:17:09Z,2015,0.689,0.481,10.0,-7.503,1.0,0.0815,...,0,0,0,0,0,0,0,0,0,chill
4,Let Me Love You,17,2021-06-20T17:17:22Z,2016,0.649,0.716,8.0,-5.371,1.0,0.0349,...,0,0,0,0,0,0,0,0,0,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Levitating,16,2023-03-01T14:45:27Z,2020,0.695,0.884,6.0,-2.278,0.0,0.0753,...,0,0,0,0,0,0,0,0,0,happy
95,Dusk Till Dawn (feat. Sia) - Radio Edit,16,2023-03-01T14:45:59Z,2018,0.259,0.437,11.0,-6.589,0.0,0.0386,...,0,0,0,0,0,0,0,0,0,sad
96,Infinity,16,2023-03-01T14:46:32Z,2017,0.671,0.673,11.0,-6.941,0.0,0.0444,...,0,0,0,0,0,0,0,0,0,chill
97,Way down We Go,17,2023-07-11T16:37:08Z,2016,0.489,0.505,10.0,-8.022,0.0,0.1170,...,0,0,0,0,0,0,0,0,0,chill


In [52]:
excel_df.to_csv('f.csv')
excel_df['artists']

0                                         [Harrdy Sandhu]
1                                    [Badshah, Payal Dev]
2                               [Badshah, Nikhita Gandhi]
3                                               [Badshah]
4       [Guru Randhawa, Nikhita Gandhi, Tanishk Bagchi...
                              ...                        
1587                                                [Nej]
1588                                           [Swattrex]
1589                                        [Roddy Ricch]
1590                                           [Dua Lipa]
1591                                   [Eminem, Dina Rae]
Name: artists, Length: 1592, dtype: object